In [2]:
import numpy as np
import math
import operator
import networkx as nx
from networkx.algorithms import approximation as approx
from itertools import permutations
import matplotlib.pyplot as plt
from deap import creator, base, tools, algorithms,gp
import dimod
import dwave_networkx as dnx
import import_ipynb
import tsp_parser
from tsp_parser import read_tsp_file
from operator import itemgetter
from IPython.display import Image 
import copy
import pickle
from tqdm import tqdm
import tsplib95
import tspsolve

In [3]:
tsp = read_tsp_file('./symmetric_problems/berlin52.tsp')

In [4]:
problem = tsplib95.load('./symmetric_problems/berlin52.tsp')
# problem = tsplib95.load('./symmetric_problems/berlin52.tsp')
G = problem.get_graph()
# tsp_problem = problem.as_keyword_dict()
# list(problem.get_nodes())

In [5]:
distance_matrix = nx.to_numpy_matrix(G)

In [6]:
problem.is_complete()

True

In [7]:
def DrawGraph(G,color):
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels = True, edge_color = color)  #with_labels=true is to show the node number in the output graph
    edge_labels = nx.get_edge_attributes(G,'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels = edge_labels,  font_size = 11) #prints weight on all the edges
    return pos

In [ ]:
def distance(A, B): 
    "The Euclidean distance between two cities."
    return math.sqrt( ((A[0]-B[0])**2)+((A[1]-B[1])**2) )

In [ ]:
def total_distance(G):
    total =0
    for edge in G.edges: 
        total += distance(G.nodes[edge[0]]["pos"],G.nodes[edge[1]]["pos"])
    return total

In [ ]:
def shortest_path(G,P):
    total =0
    for index in range(len(P)-1): 
        total += distance(G.nodes[P[index]]["pos"],G.nodes[P[index +1]]["pos"])
    return total

In [ ]:
def create_graph(cities):
    G = nx.Graph()  
    G=nx.complete_graph(len(cities))
    for index,city in enumerate(cities):
        G.nodes[index]['pos'] = (city.x,city.y)
    nx.set_edge_attributes(G, values = 1, name = 'weight')
    
    return G
    

In [ ]:
G = create_graph(tsp['CITIES'])

In [ ]:
def create_tour(individual):
    G = create_graph(tsp['CITIES'])
    G.clear_edges()
    for index,city in enumerate(tsp['CITIES']):
        G.nodes[index]['pos'] = (city.x,city.y)
        
    for index in range(len(individual)-1):
        G.add_edge(individual[index],individual[index+1])
    G.add_edge(individual[len(individual)-1],individual[0])
    
    return G

In [ ]:
def evaluation(individual):
    return (total_distance(create_tour(individual)),)

In [ ]:
def GraphCentroid(G):
    x = [G.nodes[p]['pos'][0] for p in G.nodes]
    y = [G.nodes[p]['pos'][1] for p in G.nodes]
    centroid = (sum(x) / len(G.nodes), sum(y) / len(G.nodes))
    return centroid

In [ ]:
def GraphStd(G):
    x = [G.nodes[p]['pos'][0] for p in G.nodes]
    y = [G.nodes[p]['pos'][1] for p in G.nodes]
    return np.std([x,y],axis=0)

### GENETIC PROGRAMMING

In [ ]:
pos = DrawGraph(G,'red')

In [ ]:
def plotGraph(G,show_weights=False):
    
    color_map = []
    node_sizes = []
    for node in G:
        if node == "center":
            color_map.append('red')
            node_sizes.append(800)
            
        else: 
            color_map.append('#6b61f4')
            node_sizes.append(300)

        
    pos=nx.get_node_attributes(G,'pos')
    nx.draw(G,pos,node_color=color_map,node_size = node_sizes,with_labels=True)
    labels = nx.get_edge_attributes(G,'weight')
    if show_weights:
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)

    plt.show()

In [ ]:
plotGraph(G)

In [ ]:
print(total_distance(G))

In [ ]:
def Div(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

In [ ]:
def Mod(left, right):
    try:
        return left % right
    except ZeroDivisionError:
        return 1

In [ ]:
pset = gp.PrimitiveSet("MAIN",6)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(Div, 2)
pset.addPrimitive(Mod, 2)
#Number of nodes in the graph
pset.renameArguments(ARG0="Nn")
#Number of remaining nodes to visit
pset.renameArguments(ARG1="Nrn")
#Distance from the current node
pset.renameArguments(ARG2="Dcn")
#Distance from the initial node
pset.renameArguments(ARG3="Din")
#Distance from the centroid of the nodes
pset.renameArguments(ARG4="Dc")
#Predicted distance from the initial node
pset.renameArguments(ARG5="Pd")
# #Distance left estimation
# pset.renameArguments(ARG0="Dle")

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

In [ ]:
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

In [ ]:
def euc_distance(A, B): 
    return math.sqrt( ((A[0]-B[0])**2)+((A[1]-B[1])**2))

In [ ]:
def nearest_neighbor(current,unvisited_nodes, G):
    min_node = min(unvisited_nodes, key=lambda x: euc_distance(G.nodes[x]["pos"], G.nodes[current]['pos']))
    return min_node

In [ ]:
def greedy_TSP(G,visited):
    vis = visited
    while len(vis) < len(G.nodes):
        C = nearest_neighbor(vis[-1], np.setdiff1d(list(G.nodes),vis),G)
        vis.append(C)
#     vis.append(vis[0])
    score = evaluation(vis)[0]
    return vis,score

In [ ]:
%%time
route,score = greedy_TSP(G,[0])

In [ ]:
score

In [ ]:
def pred_shortest_path(G,visited,node):
    unvisited_nodes = np.setdiff1d(list(G.nodes),visited).tolist()
    unvisited_nodes.insert(0,0)
    sub_G = G.subgraph(unvisited_nodes)
    return shortest_path(G,nx.shortest_path(sub_G,source=node,target=0))
    #return sub_G

In [ ]:
def evaluate(individual,G):
#     G.clear_edges()
    tree = gp.PrimitiveTree(individual)
    
    node_results = [0]
    current_node = 0
    initial_node =0
    visited = [0]
    solution = [0]
    unvisited = np.setdiff1d(list(G.nodes),visited)
    while len(unvisited) > 1:
        results = []
        for index,node in enumerate(unvisited):
            if index not in visited:
                Nn = len(G.nodes())
                Nrn = len(np.setdiff1d(list(G.nodes),visited))
                Dcn = distance(G.nodes[current_node]["pos"],G.nodes[node]["pos"])
                Din = distance(G.nodes[initial_node]["pos"],G.nodes[node]["pos"])
                Dc = distance(GraphCentroid(G),G.nodes[node]["pos"])
#                Pd = greedy_TSP(G,visited)
                #Pd = shortest_path(G,nx.shortest_path(G,source=node,target=0))
                Pd = pred_shortest_path(G,visited,node)
                result = toolbox.compile(expr=individual)
                results.append({"result": result(Nn,Nrn,Dcn,Din,Dc,Pd),"node":node})
        if not results:
            break
        nearest = min(results, key=lambda x:x['result'])
        solution.append(nearest["node"])
        current_node = nearest["node"]
        visited.append(nearest["node"])
        unvisited = np.setdiff1d(list(G.nodes),visited)

    solution.append(initial_node)
    return shortest_path(G,solution),

In [ ]:
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=2, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", evaluate, G= G)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

In [ ]:
%%time
evaluate(toolbox.individual(),G)

In [ ]:
fit_stats = tools.Statistics(key=operator.attrgetter("fitness.values"))
fit_stats.register('mean', np.mean)
fit_stats.register('min', np.min)

In [ ]:
pop = toolbox.population(n=50)
pop, log = algorithms.eaSimple(pop, toolbox,
                                 cxpb=0.8, mutpb=0.2,
                                 ngen=20,stats=fit_stats,verbose=True)

In [ ]:
%%time
min_results =[]
mean_results = []
for index in tqdm(range(10)):
    pop = toolbox.population(n=50)
    pop, log = algorithms.eaSimple(pop, toolbox,
                                 cxpb=0.8, mutpb=0.2,
                                 ngen=20,stats=fit_stats,verbose=True)
    min_results.append(np.min(log.select('min')))
    mean_results.append(np.min(log.select('mean')))

In [ ]:
plt.figure(figsize=(11, 4))
plt.plot(log.select('mean'),label="mean")
plt.plot(log.select('min'),label="min")
plt.legend()
plt.ylabel('Fitness'); plt.xlabel('Iterations');
plt.title("eil51 symmetric mean training runs")

In [ ]:
tree = gp.PrimitiveTree(pop[1]) 
string_tree = str(tree)
print(string_tree)

In [ ]:
mean_distance = np.mean(min_results)
min_distance = np.min(min_results)
std_distance = np.std(min_results)

In [ ]:
print("scores,mean:{},min:{},std:{}".format(mean_distance,min_distance,std_distance))